<a href="https://colab.research.google.com/github/ole12345/yolov9/blob/main/yolo_v9train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ole12345/yolov9.git

Cloning into 'yolov9'...
remote: Enumerating objects: 545, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 545 (delta 17), reused 26 (delta 7), pack-reused 499 (from 1)
Receiving objects: 100% (545/545), 2.62 MiB | 27.69 MiB/s, done.
Resolving deltas: 100% (292/292), done.


In [ ]:
%cd yolov9
!pip install -r requirements.txt --quiet
import os
os.kill(os.getpid(), 9)


/content/yolov9
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 106.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%cd yolov9
!pwd

/content/yolov9
/content/yolov9


In [ ]:
import os
from utils.filter_dataset import DatasetConfig, SubsetStrEnum, filter_dataset
import yaml

root_dir = os.getcwd()
dataset_name="coco-2017-person-ball-small"
category_list=["person","sports ball"]
yaml_content = {"path":os.path.join(".","datasets",dataset_name)}

#validation
subset = SubsetStrEnum.VALIDATION
max_number_of_images = None
cfg= DatasetConfig(root_dir = root_dir,dataset_name=dataset_name, subset = subset, max_number_of_images = max_number_of_images,category_list=category_list)
cat_id_name_val = filter_dataset(cfg)
yaml_content["val"] = subset+".txt"
#train
subset = SubsetStrEnum.TRAIN
max_number_of_images = None
cfg= DatasetConfig(root_dir = root_dir,dataset_name=dataset_name, subset = subset, max_number_of_images = max_number_of_images,category_list=category_list)
cat_id_name_train = filter_dataset(cfg)
yaml_content["train"] = subset+".txt"
if cat_id_name_train!=cat_id_name_val:
   raise Exception("Inconsistent categories")

#write the yaml file
yaml_content["names"] = cat_id_name_train
yaml_content["stuff_names"] = list()
yaml_content["download"] = None
with open(os.path.join(root_dir,"data",dataset_name+".yaml"),"w") as file:
   yaml.dump(yaml_content, file, default_flow_style=False)
   file.close()

annotations_trainval2017.zip: 100%|██████████| 241M/241M [00:22<00:00, 11.3MiB/s]


loading annotations into memory...
Done (t=0.57s)
creating index...
index created!
Selected 2522 images of 'instances_val2017.json' with following class distribution:
'      person' : 2512 images
' sports ball' : 169 images


KeyboardInterrupt: 

In [ ]:
!python segment/train.py --device 0 --batch 32  --data coco-2017-person-ball-small.yaml --img 640 --cfg models/segment/gelan-c-seg-test.yaml --weights '' --name test_model --hyp hyp.scratch-high.yaml --no-overlap --epochs 1 --close-mosaic 10

In [ ]:
# prompt: I want to download several images in parallel with a nice progress bar

import os
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

def download_image(url, save_path):
  try:
    response = requests.get(url, stream=True)
    response.raise_for_status()
    with open(save_path, 'wb') as f:
      for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)
    return True
  except requests.exceptions.RequestException as e:
    print(f"Error downloading {url}: {e}")
    return False

def download_images_parallel(image_urls, save_dir):
  os.makedirs(save_dir, exist_ok=True)
  with ThreadPoolExecutor() as executor:
    futures = []
    for i, url in enumerate(image_urls):
      save_path = os.path.join(save_dir, f"image_{i}.jpg")
      futures.append(executor.submit(download_image, url, save_path))

    with tqdm(total=len(image_urls), desc="Downloading images") as pbar:
      for future in futures:
        if future.result():
          pbar.update(1)


# Example usage:
image_urls = [
    "https://images.unsplash.com/photo-1682687214223-d3980c271543?ixlib=rb-4.0.3&ixid=MnwxMjA3fDF8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1170&q=80",
    "https://images.unsplash.com/photo-1682685797398-4a96969d3232?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1170&q=80",
    "https://images.unsplash.com/photo-1682685797612-748f8d730458?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1170&q=80",
]

save_dir = "downloaded_images"
download_images_parallel(image_urls, save_dir)
